# SyGMA processing for metabolites

Info: Jupyter notebook made by LF and Pryia on April 2018

Description: Process a list of structures (InChI) and generate Phase I and II metabolites

In [1]:
import pandas as pd
import numpy as np
import sygma
from rdkit import Chem

In [2]:
#Open dataframes
df = pd.read_csv("Saliva_InChI.txt",  sep='\t', header=0, encoding='utf-8')
df2 = pd.read_csv("drugbank_approved_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df3 = pd.read_csv("drugbank_illicit_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df4 = pd.read_csv("drugbank_nutraceutical_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df5 = pd.read_csv("FOODB_compounds_InChI.txt",  sep='\t', header=0, encoding='utf-8')
df6 = pd.read_csv("Exposome_explorer_biomarkers.csv",  sep=',', header=0)
df7 = pd.read_csv("toxins_edited_v2.txt",  sep='\t', header=0, encoding='latin-1')
df8 = pd.read_csv("PTID_Pesticide_3D-Plant2Cells_compound.txt",  sep='\t', header=0, encoding='latin-1')

In [3]:
#Keep just a column per dataset
df = df[['InChI']]
df2 = df2[['InChI']]
df3 = df3[['InChI']]
df4 = df4[['InChI']]
df5 = df5[['InChI']]
df6 = df6[['InChI']]
df7 = df7[['InChI Identifier']]
df7.rename(columns={'InChI Identifier': 'InChI'})
df8 = df8[['InChI']]

In [4]:
#Concatenate table
frames = [df, df2, df3, df4, df5, df6, df7, df8]

Table = pd.concat(frames)

In [ ]:
# Write concatenated table parent compounds
Table_concatenated = Table[['SMILES']]

In [5]:
#Unique count of InChI
len(Table.InChI.unique())

11627

In [6]:
#Remove empty rows
Table_clean = Table[Table.InChI.str.contains('nan') == False]

#Remove non-Carbonated molecules
Table_clean = Table_clean[Table_clean.InChI.str.startswith('InChI=1S/C')]

#Filter SMILES and Formula lenght
Table_clean = Table_clean[Table_clean['InChI'].map(len) > 20]
Table_clean = Table_clean[Table_clean['InChI'].map(len) < 250]

#Sort by InChI lenght
Table_clean.sort_values('InChI',inplace=True, ascending=False)
Table_clean.index = Table_clean['InChI'].str.len()
Table_clean = Table_clean.sort_index(ascending=False).reset_index(drop=True)
Table_clean.shape

(10868, 2)

In [ ]:
#Write the parent file out
Table_clean.to_csv('Combined_DB_DrugDB_FoodDB_Pesticide_Exposome_Sygma_InChI.tsv', sep = '\t', index = False)

In [7]:
#Make full list from SMILES column
List_inchi = []
List_inchi = Table_clean['InChI'].tolist()

#Remove duplicates
List_inchi_unique = []
for item in List_inchi:
    if item not in List_inchi_unique:
        List_inchi_unique.append(item)
        
print(len(List_inchi_unique))

9516


## Running SyGMa

Each step in a scenario lists the ruleset
and the number of reaction cycles to be applied

In [8]:
# Define SyGMA scenario. 
scenario = sygma.Scenario([
    [sygma.ruleset['phase1'], 1],
    [sygma.ruleset['phase2'], 1]])

In [9]:
# Create a list of RDkit object from smiles
list_mol = [Chem.MolFromInchi(x) for x in List_inchi_unique]
list_mol

 None,
 None,
 None,
 None,
 ...]

In [10]:
# Remove None
list_mol_clean = [x for x in list_mol if x!=None]

In [11]:
print(len(list_mol))
print(len(list_mol_clean))

9516
9451


In [12]:
data = pd.DataFrame(data=None)
data2 = pd.DataFrame(data=None)
metabolites = []
for x in list_mol_clean:
    print(x)
    metabolic_tree = scenario.run(x)
    metabolic_tree.calc_scores()
    metabolites = metabolic_tree.to_smiles()
    data = pd.DataFrame(metabolites[1:],columns=metabolites[0])
    data['parent'] = (metabolites[0][0])
    data.columns.values[0] = 'metabolite'
    data.columns.values[1] = 'score'
    #Keep only top20
    data2 = data2.append(data[:10], ignore_index=True)

ZeroDivisionError: float division by zero

In [ ]:
metabolites

In [ ]:
data2.shape

In [ ]:
#Number of unique parent
len(data2.metabolite.unique())

In [ ]:
data2.head(5)

In [120]:
#Write the file out
data2.to_csv('Combined_DB_DrugDB_FoodDB_Pesticide_Exposome_Saliva_Sygma_1_1_metabolites_TOP10.tsv', sep = '\t', index = False)